# Lesson: Data Labelling and Classification using gpt-4o-mini

### Starting up the code, getting env key

In [2]:
import pandas as pd
import os
from openai import OpenAI
from dotenv import load_dotenv

# 1. Setup API Key from .env
load_dotenv()
client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))

### Loading the CSV file and reading it

In [3]:
df = pd.read_csv('skincare_reviews.csv')

# printing first few rows to verify that it works
print("Input Data:")
print(df.head())

Input Data:
                                              review
0  Produk ini sangat melembapkan dan membuat kuli...
1  Teksturnya ringan dan cepat meresap tanpa rasa...
2  Setelah pemakaian rutin kulit terlihat lebih c...
3  Aman untuk kulit sensitif dan tidak menimbulka...
4  Hasilnya memuaskan dan sesuai dengan klaim pro...


### Prompt to extract info and label the dataset

In [4]:
def get_sentiment(text):
    """Function to call OpenAI API for labeling"""
    if pd.isna(text):
        return "N/A"
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are a data labeling expert. "
                "Label the sentiment of the following skincare review as 'Positive', 'Neutral', or 'Negative'. "
                "Return ONLY the word."},
                {"role": "user", "content": text}
            ],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

### Apply the labeling function to the DataFrame

In [5]:
print("Labelling data...")
df['label'] = df['review'].apply(get_sentiment)

display(df) # shows the new DF with labels from OpenAI

Labelling data...


,review,label
0,Produk ini sangat melembapkan dan membuat kuli...,Positive
1,Teksturnya ringan dan cepat meresap tanpa rasa...,Positive
2,Setelah pemakaian rutin kulit terlihat lebih c...,Positive
3,Aman untuk kulit sensitif dan tidak menimbulka...,Positive
4,Hasilnya memuaskan dan sesuai dengan klaim pro...,Positive
5,Produk ini menyebabkan kulit saya beruntusan.,Negative
6,Terasa perih dan membuat kulit menjadi kemerahan.,Negative
7,Tidak memberikan efek apa pun meskipun sudah d...,Negative
8,Teksturnya terlalu berat dan membuat wajah ber...,Negative
9,Produk ini justru memperparah jerawat saya.,Negative


## Saving the results into labelled_skincare_reviews.csv

In [6]:
df.to_csv('labelled_skincare_reviews.csv', index=False)
print("Done! Results saved to 'labelled_skincare_reviews.csv'")

Done! Results saved to 'labelled_skincare_reviews.csv'


## Next Step: Classification of Reviews by their Type

In [7]:
def get_data_type(text):
    """Function to categorize the review topic"""
    if pd.isna(text):
        return "N/A"
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {
                    "role": "system", 
                    "content": (
                        "You are a skincare analyst. Classify the following review into one of these types: "
                        "'Texture' (feel, weight, absorption), "
                        "'Effectiveness' (results, hydration, brightening), "
                        "or 'Side Effects' (irritation, redness, acne). "
                        "Return ONLY the category name."
                    )
                },
                {"role": "user", "content": text}
            ],
            temperature=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        return f"Error: {e}"

### showing fully labelled DF

In [8]:
print("Classifying data types...")
df['data_type'] = df['review'].apply(get_data_type)

display(df) # shows the new DF with data types from OpenAI

Classifying data types...


,review,label,data_type
0,Produk ini sangat melembapkan dan membuat kuli...,Positive,Effectiveness
1,Teksturnya ringan dan cepat meresap tanpa rasa...,Positive,Texture
2,Setelah pemakaian rutin kulit terlihat lebih c...,Positive,Effectiveness
3,Aman untuk kulit sensitif dan tidak menimbulka...,Positive,Error: Error code: 429 - {'error': {'message':...
4,Hasilnya memuaskan dan sesuai dengan klaim pro...,Positive,Effectiveness
5,Produk ini menyebabkan kulit saya beruntusan.,Negative,Side Effects
6,Terasa perih dan membuat kulit menjadi kemerahan.,Negative,Side Effects
7,Tidak memberikan efek apa pun meskipun sudah d...,Negative,Effectiveness
8,Teksturnya terlalu berat dan membuat wajah ber...,Negative,Texture
9,Produk ini justru memperparah jerawat saya.,Negative,Side Effects


### Saving the final version with the classification of data types

In [9]:
# Save the final version
df.to_csv('fully_labeled_skincare_reviews.csv', index=False)
print("Final results saved to 'fully_labeled_skincare_reviews.csv'")

Final results saved to 'fully_labeled_skincare_reviews.csv'
